[![](images/colab-badge.png)](https://colab.research.google.com/github/fzampirolli/si-md2/blob/main/si-md2/notebooks_alunos/cap02/cap02_aluno.ipynb)


# Mineração de Dados e Regras de Associação

## Introdução

A Mineração de Dados é uma disciplina tão vasta que qualquer publicação sobre o tema obriga o autor a selecionar alguns tópicos em detrimento de outros não menos importantes. A atividade de **Regras de Associação** foi o tópico escolhido para iniciarmos a apresentação das principais tarefas da Mineração de Dados por envolver ideias bem intuitivas. A analogia entre Regras de Associação e Cesta de Compras facilita o entendimento de como descobrir padrões de associação entre itens de um conjunto qualquer.

## Mineração de Dados

Durante o processo de **Descoberta de Conhecimento em Bases de Dados**, **KDD**, é na etapa de **Mineração de Dados** que efetivamente são encontrados os padrões de associação implícitos nos dados. A análise automatizada dessa massa de dados visa detectar regularidades, ou quebra de regularidade, que constitui informação implícita, porém supostamente desconhecida, e útil para determinado fim.

A Mineração de Dados pode ser vista como a sistematização de teorias, técnicas e algoritmos desenvolvidos em outras disciplinas já consagradas, como a Estatística, a Inteligência Artificial, o Aprendizado de Máquina, a Base de Dados etc. (@fig-2-1). O propósito da Mineração de Dados é detectar automaticamente padrões de associação úteis e não óbvios em grandes quantidades de dados.

No dia a dia, uma quantidade incalculável de dados é gerada na forma de registros de vendas, textos brutos, imagens, sons, gráficos etc., tanto por sistemas computacionais como por seres humanos, constituindo uma espécie de informação não estruturada. Embora esta forma de registro de dados seja adequada para o ser humano, quando se trata de analisar grandes quantidades de dados de forma automatizada, é comum e conveniente que se introduza alguma **estrutura** que facilite o acesso e o processamento sistemático.

![A Relação da Mineração de Dados com Algumas Disciplinas Correlatas.](images/fig2_1.png){#fig-2-1 width=70%}

Para que parte de toda essa informação não estruturada possa ser utilizada na Mineração de Dados, geralmente é feita uma seleção e um pré-processamento visando transformar dados brutos em coleções estruturadas de dados. Em termos práticos, para nossas considerações iniciais, a estrutura de representação de uma Base de Dados pode ser semelhante a uma tabela de dados, sendo cada linha dessa tabela uma **transação** ou um **exemplo**. Cada **transação** é composta por um ou mais **itens** ou, visto de outra forma, cada **exemplo** é caracterizado por seus **atributos**.


As @tbl-2-cestas, @tbl-2-booleana e @tbl-2-tempo ilustram formas de dados estruturados convenientes para a Mineração de Dados.

In [25]:
#| quarto-raw: true
#| label: tbl-2-cestas
#| tbl-cap: "Cestas de Compras"
#| echo: false
#| output: asis

import pandas as pd
from IPython.display import Markdown

dados = {
    'TID': [1, 2, 3, 4, 5],
    'Itens': [
        '{Arroz, Feijão, Óleo}',
        '{Queijo, Vinho}',
        '{Arroz, Feijão, Batata, Óleo}',
        '{Arroz, Água, Queijo, Vinho}',
        '{Arroz, Feijão, Batata, Óleo}'
    ]
}
df = pd.DataFrame(dados)

# Para garantir que o Quarto entenda como Markdown puro e o Colab mostre formatado:
Markdown(df.to_markdown(index=False, colalign=("center", "left")))

|  TID  | Itens                         |
|:-----:|:------------------------------|
|   1   | {Arroz, Feijão, Óleo}         |
|   2   | {Queijo, Vinho}               |
|   3   | {Arroz, Feijão, Batata, Óleo} |
|   4   | {Arroz, Água, Queijo, Vinho}  |
|   5   | {Arroz, Feijão, Batata, Óleo} |

Na @tbl-2-cestas cada uma das Transações possui uma IDentificação (TID), e seus itens representam artigos vendidos em um supermercado. Se a tabela de itens for muito extensa, como costuma ser em casos reais, pode ser ainda mais conveniente representar cada um de seus itens na forma de um atributo associado a um valor booleano, como mostra a @tbl-2-booleana.


In [26]:
#| quarto-raw: true
#| label: tbl-2-booleana
#| tbl-cap: "Representação Booleana de Cestas de Compras"
#| echo: false

import pandas as pd
from IPython.display import Markdown

dados_booleana = {
    'TID': [1, 2, 3, 4, 5],
    'Arroz': ['y', 'n', 'y', 'y', 'y'],
    'Feijão': ['y', 'n', 'y', 'n', 'y'],
    'Batata': ['n', 'n', 'y', 'n', 'y'],
    'Óleo': ['y', 'n', 'y', 'n', 'y'],
    'Água': ['n', 'n', 'n', 'y', 'n'],
    'Queijo': ['n', 'y', 'n', 'y', 'n'],
    'Vinho': ['n', 'y', 'n', 'y', 'n']
}
df_booleana = pd.DataFrame(dados_booleana)

Markdown(df_booleana.to_markdown(index=False, colalign=("center",) * len(df_booleana.columns)))

|  TID  |  Arroz  |  Feijão  |  Batata  |  Óleo  |  Água  |  Queijo  |  Vinho  |
|:-----:|:-------:|:--------:|:--------:|:------:|:------:|:--------:|:-------:|
|   1   |    y    |    y     |    n     |   y    |   n    |    n     |    n    |
|   2   |    n    |    n     |    n     |   n    |   n    |    y     |    y    |
|   3   |    y    |    y     |    y     |   y    |   n    |    n     |    n    |
|   4   |    y    |    n     |    n     |   n    |   y    |    y     |    y    |
|   5   |    y    |    y     |    y     |   y    |   n    |    n     |    n    |

Um exemplo clássico de uma Base de Dados usada em artigos sobre Mineração de Dados é apresentada na @tbl-2-tempo  (@quinlan1986 *apud* @witten2005), composta por dados fictícios sobre as condições de tempo para que ocorra ou não a partida de um esporte não especificado. A tabela é composta por 14 exemplos (linhas), cada um com cinco atributos (colunas): Dia, Temperatura, Umidade, Vento e Partida. A tabela pode também ser interpretada de outra forma, como sendo composta por quatro atributos (Dia, Temperatura, Umidade e Vento) e uma classe (Partida), que representa o resultado da combinação dos quatro atributos.

In [27]:
#| quarto-raw: true
#| label: tbl-2-tempo
#| tbl-cap: "Tabela do Tempo"
#| echo: false

import pandas as pd
from IPython.display import Markdown

dados_tempo = {
    'Dia': ['Ensolarado', 'Ensolarado', 'Nublado', 'Chuvoso', 'Chuvoso', 'Chuvoso', 'Nublado', 'Ensolarado', 'Ensolarado', 'Chuvoso', 'Ensolarado', 'Nublado', 'Nublado', 'Chuvoso'],
    'Temperatura': ['Elevada', 'Elevada', 'Elevada', 'Amena', 'Baixa', 'Baixa', 'Baixa', 'Amena', 'Baixa', 'Amena', 'Amena', 'Amena', 'Elevada', 'Amena'],
    'Umidade': ['Alta', 'Alta', 'Alta', 'Alta', 'Normal', 'Normal', 'Normal', 'Alta', 'Normal', 'Normal', 'Normal', 'Alta', 'Normal', 'Alta'],
    'Vento': ['Falso', 'Verdadeiro', 'Falso', 'Falso', 'Falso', 'Verdadeiro', 'Verdadeiro', 'Falso', 'Falso', 'Falso', 'Verdadeiro', 'Verdadeiro', 'Falso', 'Verdadeiro'],
    'Partida': ['Não', 'Não', 'Sim', 'Sim', 'Sim', 'Não', 'Sim', 'Não', 'Sim', 'Sim', 'Sim', 'Sim', 'Sim', 'Não']
}
df_tempo = pd.DataFrame(dados_tempo)

Markdown(df_tempo.to_markdown(index=False))

| Dia        | Temperatura   | Umidade   | Vento      | Partida   |
|:-----------|:--------------|:----------|:-----------|:----------|
| Ensolarado | Elevada       | Alta      | Falso      | Não       |
| Ensolarado | Elevada       | Alta      | Verdadeiro | Não       |
| Nublado    | Elevada       | Alta      | Falso      | Sim       |
| Chuvoso    | Amena         | Alta      | Falso      | Sim       |
| Chuvoso    | Baixa         | Normal    | Falso      | Sim       |
| Chuvoso    | Baixa         | Normal    | Verdadeiro | Não       |
| Nublado    | Baixa         | Normal    | Verdadeiro | Sim       |
| Ensolarado | Amena         | Alta      | Falso      | Não       |
| Ensolarado | Baixa         | Normal    | Falso      | Sim       |
| Chuvoso    | Amena         | Normal    | Falso      | Sim       |
| Ensolarado | Amena         | Normal    | Verdadeiro | Sim       |
| Nublado    | Amena         | Alta      | Verdadeiro | Sim       |
| Nublado    | Elevada       | Normal    | Falso      | Sim       |
| Chuvoso    | Amena         | Alta      | Verdadeiro | Não       |

Uma análise mais atenta dos exemplos das @tbl-2-cestas, @tbl-2-booleana mostra que alguns desses atributos sempre aparecem juntos e que, portanto, várias **Regras de Associação** podem ser extraídas dessas tabelas.

## Regras de Associação 

## Referências do Capítulo

Tem que citar neste capítulo: @han2008, @tan2009, @witten2005, @goldschmidt2005

\printbibliography[heading=none]
